In [57]:
import numpy
import opt_einsum as oe

naux = 4000
nocc = 100
nvir = 1000
xx = [numpy.zeros(x) for x in (
    (naux, nvir), (naux, nvir),
    (naux, nvir), (naux, nvir),
    (naux, naux),
    (nvir, nvir, nocc, nocc),
    (nvir, nvir, nocc, nocc))
]

my_expr = oe.contract_path(
    "Pa,Pc,Qb,Qd,PQ,dcji->abij", *xx
)
print(my_expr)

([(0, 1), (0, 1), (0, 3), (2, 3), (0, 2), (0, 1)],   Complete contraction:  Pk,Pc,Ql,Qd,PQ,acli,bdkj->abij
         Naive scaling:  10
     Optimized scaling:  6
      Naive FLOP count:  1.120e+28
  Optimized FLOP count:  4.083e+15
   Theoretical speedup:  2.743e+12
  Largest intermediate:  1.000e+10 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   3              0             Pc,Pk->Pck          Ql,Qd,PQ,acli,bdkj,Pck->abij
   3              0             Qd,Ql->Qdl            PQ,acli,bdkj,Pck,Qdl->abij
   4           GEMM            Pck,PQ->ckQ               acli,bdkj,Qdl,ckQ->abij
   5           GEMM          ckQ,Qdl->ckdl                  acli,bdkj,ckdl->abij
   6           TDOT        ckdl,acli->kdai                       bdkj,kdai->abij
   6           TDOT        kdai,bdkj->abi

In [58]:
xx = [numpy.zeros(x) for x in (
    (naux, nvir, nvir), 
    (naux, nvir, nvir),
    (nvir, nvir, nocc, nocc))
]

ss = ['Pac,Pbd,dcji->abij', 'Pbc,Pad,dcji->abij']
for s in ss:
    my_expr = oe.contract_path(s, *xx)[1]
    print(my_expr)
    print()

xx = [numpy.zeros(x) for x in (
    (naux, nvir, nvir), 
    (naux, nvir, nvir),
    (nvir, nocc), (nvir, nocc))
]
ss  = ['Pac,Pbd,ci,dj->abij', 'Pac,Pbd,di,cj->abij']
ss += ['Pbc,Pad,ci,dj->abij', 'Pbc,Pad,di,cj->abij']
for s in ss:
    my_expr = oe.contract_path(s, *xx, optimize="optimal")
    print(my_expr)
    print()

  Complete contraction:  Pac,Pbd,dcji->abij
         Naive scaling:  7
     Optimized scaling:  6
      Naive FLOP count:  1.200e+20
  Optimized FLOP count:  2.800e+16
   Theoretical speedup:  4.286e+3
  Largest intermediate:  1.000e+12 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   5           GEMM          Pbd,Pac->bdac                       dcji,bdac->abij
   6           TDOT        bdac,dcji->abij                            abij->abij

  Complete contraction:  Pbc,Pad,dcji->abij
         Naive scaling:  7
     Optimized scaling:  6
      Naive FLOP count:  1.200e+20
  Optimized FLOP count:  2.800e+16
   Theoretical speedup:  4.286e+3
  Largest intermediate:  1.000e+12 elements
--------------------------------------------------------------------------------
scaling        BLAS  

In [59]:
xx = [numpy.zeros(x) for x in (
    (naux, nocc, nvir), (naux, nocc, nvir),
    (nvir, nvir, nocc, nocc),
    (nvir, nvir, nocc, nocc))
]

ss = [
    'Pkc,Pld,acji,bdlk->abij',
    'Pkc,Pld,acki,bdlj->abij',
    'Pkc,Pld,acli,bdkj->abij',
    'Pkc,Pld,adji,bclk->abij',
    'Pkc,Pld,adki,bclj->abij',
    'Pkc,Pld,adli,bckj->abij',
    'Pkc,Pld,baki,dclj->abij',
    'Pkc,Pld,bali,dckj->abij',
    'Pkc,Pld,bcji,adlk->abij',
    'Pkc,Pld,bcki,adlj->abij',
    'Pkc,Pld,bcli,adkj->abij',
    'Pkc,Pld,bdji,aclk->abij',
    'Pkc,Pld,bdki,aclj->abij',
    'Pkc,Pld,bdli,ackj->abij',
    'Pkc,Pld,dcji,balk->abij',
    'Pkc,Pld,dcki,balj->abij',
    'Pkc,Pld,dcli,bakj->abij',
]

for s in ss:
    my_expr = oe.contract_path(s, *xx, optimize="optimal")[1]
    print(my_expr)
    print()

  Complete contraction:  Pkc,Pld,acji,bdlk->abij
         Naive scaling:  9
     Optimized scaling:  5
      Naive FLOP count:  1.600e+24
  Optimized FLOP count:  1.008e+14
   Theoretical speedup:  1.587e+10
  Largest intermediate:  1.000e+10 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   5           TDOT          bdlk,Pld->bkP                    Pkc,acji,bkP->abij
   4           TDOT            bkP,Pkc->bc                         acji,bc->abij
   5           TDOT          bc,acji->abij                            abij->abij

  Complete contraction:  Pkc,Pld,acki,bdlj->abij
         Naive scaling:  9
     Optimized scaling:  5
      Naive FLOP count:  1.600e+24
  Optimized FLOP count:  2.400e+14
   Theoretical speedup:  6.667e+9
  Largest intermediate:  1.000e+10 elements
----------

In [60]:
xx = [numpy.zeros(x) for x in (
    (nocc, nocc, nvir, nvir),
    (nvir, nvir, nocc, nocc),
    (nvir, nvir, nocc, nocc))
]

ss = [
    'klcd,acki,bdlj->abij',
    'klcd,adji,bclk->abij',
    'klcd,adki,bclj->abij',
    'klcd,baki,dclj->abij',
    'klcd,bcki,adlj->abij',
    'klcd,bdji,aclk->abij',
    'klcd,bdki,aclj->abij',
    'klcd,dcji,balk->abij',
    'klcd,dcki,balj->abij'
]

for s in ss:
    my_expr = oe.contract_path(s, *xx, optimize="optimal")[1]
    print(my_expr)
    print()

  Complete contraction:  klcd,acki,bdlj->abij
         Naive scaling:  8
     Optimized scaling:  6
      Naive FLOP count:  3.000e+20
  Optimized FLOP count:  4.000e+15
   Theoretical speedup:  7.500e+4
  Largest intermediate:  1.000e+10 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   6           TDOT        acki,klcd->aild                       bdlj,aild->abij
   6           TDOT        aild,bdlj->abij                            abij->abij

  Complete contraction:  klcd,adji,bclk->abij
         Naive scaling:  8
     Optimized scaling:  5
      Naive FLOP count:  3.000e+20
  Optimized FLOP count:  4.000e+13
   Theoretical speedup:  7.500e+6
  Largest intermediate:  1.000e+10 elements
--------------------------------------------------------------------------------
scaling        BL

In [61]:


xx = [numpy.zeros(x) for x in (
    (nvir, nvir, nvir, nvir),
    (nvir, nvir, nocc, nocc))
]

ss = [
    'bacd,dcji->abij'
]

for s in ss:
    my_expr = oe.contract_path(s, *xx, optimize="optimal")[1]
    print(my_expr)
    print()

  Complete contraction:  bacd,dcji->abij
         Naive scaling:  6
     Optimized scaling:  6
      Naive FLOP count:  2.000e+16
  Optimized FLOP count:  2.000e+16
   Theoretical speedup:  1.000e+0
  Largest intermediate:  1.000e+10 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   6           TDOT        dcji,bacd->abij                            abij->abij



In [62]:
xx = [numpy.zeros(x) for x in (
    (nvir, nvir, nvir, nvir),
    (nvir, nocc), (nvir, nocc))
]

ss = [
    'bacd,di,cj->abij',
    'bacd,ci,dj->abij'
]

for s in ss:
    my_expr = oe.contract_path(s, *xx, optimize="optimal")[1]
    print(my_expr)
    print()

  Complete contraction:  bacd,di,cj->abij
         Naive scaling:  6
     Optimized scaling:  5
      Naive FLOP count:  3.000e+16
  Optimized FLOP count:  2.200e+14
   Theoretical speedup:  1.364e+2
  Largest intermediate:  1.000e+11 elements
--------------------------------------------------------------------------------
scaling        BLAS                current                             remaining
--------------------------------------------------------------------------------
   5           GEMM          di,bacd->ibac                         cj,ibac->abij
   5           GEMM          ibac,cj->abij                            abij->abij

  Complete contraction:  bacd,ci,dj->abij
         Naive scaling:  6
     Optimized scaling:  5
      Naive FLOP count:  3.000e+16
  Optimized FLOP count:  2.200e+14
   Theoretical speedup:  1.364e+2
  Largest intermediate:  1.000e+11 elements
--------------------------------------------------------------------------------
scaling        BLAS      